In [1]:
# %load_ext tensorboard

In [2]:
import numpy as np
import tensorflow as tf
from copy import deepcopy

import os

import datetime

import import_ipynb
from QBert import train_utils, models

from tqdm.notebook import tqdm



importing Jupyter notebook from C:\Users\LGCNS\Documents\GitHub\Q_Bert\QBert\train_utils.ipynb
importing Jupyter notebook from C:\Users\LGCNS\Documents\GitHub\Q_Bert\QBert\models.ipynb


## 해당 파일은 bert.run_pretraining.run_bert_pretrain을 구현하는 것을 목표로 한다.

 - Parameter는 FLAG 형식에서 직접 정의해주는 방식으로 변경하고, Main에서 직접 정의하도록 한다.

## Strategy

In [3]:
tf.config.list_physical_devices() # device 확인

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [4]:
# Strategy 정의

distribution_strategy = 'mirrored' # 'tpu'
num_gpus = 0
all_reduce_alg = None

if distribution_strategy == 'tpu' :
    tpu_address = ""
else :
    tpu_address = None



strategy = train_utils.get_distribution_strategy(
                  distribution_strategy=distribution_strategy,
                  num_gpus=num_gpus,
                  all_reduce_alg=all_reduce_alg,
                  tpu_address=tpu_address)

print ('\n장치의 수: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)

장치의 수: 1


## Bert_Config

In [5]:
# sub_model (core_model 필요 Config)

vocab_size = 32000 # 
hidden_size = 768 # Transformer hidden Layers
type_vocab_size = 12 #: The number of types that the 'type_ids' input can take.
num_layers = 12
num_attention_heads = 12
max_seq_length = 256 # 512
dropout_rate = .1
# attention_dropout_rate = .1
inner_dim = 3072
# hidden_act = 'gelu'
initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02)

# Pretrain Model 필요 Config
max_predictions_per_seq = 40


## Input_Files

In [6]:
filenames = ['./Test_Examples.tfrecords']

# Create a description of the features.
feature_description = {
    'input_ids': tf.io.FixedLenFeature([max_seq_length], tf.int64),
    'segment_ids': tf.io.FixedLenFeature([max_seq_length], tf.int64),
    'input_mask': tf.io.FixedLenFeature([max_seq_length], tf.int64),
    'masked_lm_positions': tf.io.FixedLenFeature([max_predictions_per_seq], tf.int64),
    'masked_lm_ids': tf.io.FixedLenFeature([max_predictions_per_seq], tf.int64),
    'masked_lm_weights': tf.io.FixedLenFeature([max_predictions_per_seq], tf.float32),
    'next_sentence_labels': tf.io.FixedLenFeature([1], tf.int64),
}

# keys = feature_description.keys()

In [7]:
def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

def _select_data_from_record(record):
    """Filter out features to use for pretraining."""
    x = {
        'input_ids': record['input_ids'],
        'input_mask': record['input_mask'],
        'segment_ids': record['segment_ids'],
        'masked_lm_positions': record['masked_lm_positions'],
        'masked_lm_ids': record['masked_lm_ids'],
        'masked_lm_weights': record['masked_lm_weights'],
    }
    if use_next_sentence_label:
        x['next_sentence_labels'] = record['next_sentence_labels']
    if use_position_id:
        x['position_ids'] = record['position_ids']

    # TODO(hongkuny): Remove the fake labels after migrating bert pretraining.
    if output_fake_labels:
        return (x, record['masked_lm_weights'])
    else:
        return x


In [19]:
tf.math.ceil

<module 'numpy' from 'C:\\Users\\LGCNS\\AppData\\Roaming\\Python\\Python37\\site-packages\\numpy\\__init__.py'>

In [21]:

BUFFER_SIZE = 10000

GLOBAL_BATCH_SIZE = 32
BATCH_SIZE_PER_REPLICA = np.ceil(GLOBAL_BATCH_SIZE // strategy.num_replicas_in_sync)

use_next_sentence_label = True
output_fake_labels = True
use_position_id = False

batch_size = 8
lr = 1e-3

filenames = ['./Test_Examples.tfrecords']

In [24]:

with strategy.scope():

    train_dataset = tf.data.Dataset.from_tensor_slices(filenames)
    train_dataset = train_dataset.interleave(tf.data.TFRecordDataset, cycle_length = -1)
    
    dataset_inputs = train_dataset.map(_parse_function) # String to Example
    dataset_inputs_with_labels = dataset_inputs.map(_select_data_from_record) # Example to InputData
    ## 본래대로라면 그냥 써도 되지만, 현재 Label이 없는 데이터이기 때문에
    ## max_predictions_per_seq 길이의 허위 정답 (Fake_y)를 삽입하는 mapping function이다.
    
    dataset = dataset_inputs_with_labels
    dataset = dataset.cache()
    dataset = dataset.shuffle(10000, reshuffle_each_iteration = True)
    dataset = dataset.batch(GLOBAL_BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    dist_dataset = strategy.experimental_distribute_dataset(dataset)

In [34]:
?strategy.experimental_distribute_dataset

## Traning Config

In [25]:
# callback

## model checkpoint
t = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

checkpoint_dir = './training_checkpoints_{}'.format(t)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

model_cp = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True),

## Learning Rate Print

class PrintLR(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print('\n에포크 {}의 학습률은 {}입니다.'.format(epoch + 1,
                                                      model.optimizer.lr.numpy()))

callbacks = [
    model_cp,
    PrintLR()
]

In [26]:
# optimizer

## Learning Rate Decay

# lr = 1e-4 warmup stage (step <= 10000)
# Decay linearly

init_lr = 1e-4
warmup_steps = 10000
num_train_steps = 1000000
end_lr = 0

lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
      initial_learning_rate=init_lr,
      decay_steps=num_train_steps,
      end_learning_rate=end_lr)

lr_schedule = train_utils.WarmUp(
        initial_learning_rate=init_lr,
        decay_schedule_fn=lr_schedule,
        warmup_steps=warmup_steps)

optimizer = train_utils.AdamWeightDecay( 
    learning_rate=lr_schedule,
    weight_decay_rate=0.01,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-6,
    exclude_from_weight_decay=['LayerNorm', 'layer_norm', 'bias'])

In [27]:
# loss
def loss_fn(fake_y, losses, **unused_args) :
    
    return tf.reduce_mean(losses, axis = -1)
    

## Model Define

In [28]:
with strategy.scope() :

    model, sub_model = models.get_bert_models_fn(vocab_size
                                             , hidden_size
                                             , type_vocab_size
                                             , num_layers
                                             , num_attention_heads
                                             , max_seq_length
                                             , max_predictions_per_seq
                                             , dropout_rate
                                             , inner_dim 
                                             , initializer)
    model.compile(optimizer, loss=loss_fn)

# 훈련

In [32]:
    model.fit(dist_dataset,
          epochs=1, #40,
          callbacks=callbacks,
          steps_per_epoch=10)#25000)

ValueError: Failed to find data adapter that can handle input: <class 'tensorflow.python.distribute.input_lib.DistributedDataset'>, <class 'NoneType'>